**Imports & Connection**

In [80]:
import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine

# Connect to the SQLite Database
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))
DB_PATH = os.path.join(BASE_DIR, 'data', 'retail.db')
engine = create_engine(f'sqlite:///{DB_PATH}')

# Load data using SQL
query = "SELECT * FROM transactions"
df = pd.read_sql(query, engine)
print(f'Load {df.shape[0]} rows.')

Load 541910 rows.


### Basic Analysis

In [81]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00.000000,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00.000000,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00.000000,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00.000000,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00.000000,3.39,17850,United Kingdom


In [82]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Invoice      541910 non-null  str    
 1   StockCode    541910 non-null  str    
 2   Description  540456 non-null  str    
 3   Quantity     541910 non-null  int64  
 4   InvoiceDate  541910 non-null  str    
 5   Price        541910 non-null  float64
 6   Customer ID  406830 non-null  str    
 7   Country      541910 non-null  str    
dtypes: float64(1), int64(1), str(6)
memory usage: 33.1 MB


In [83]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
Price               0
Customer ID    135080
Country             0
dtype: int64

In [84]:
print(f'Total number of duplicate is {df.duplicated().sum()}')

Total number of duplicate is 5268


In [85]:
df.describe()

,Quantity,Price
count,541910.000000,541910.000000
mean,9.552234,4.611138
std,218.080957,96.759765
min,-80995.000000,-11062.060000
25%,1.000000,1.250000
50%,3.000000,2.080000
75%,10.000000,4.130000
max,80995.000000,38970.000000


In [86]:
print(f"There are {len(df['Country'].unique())} unique countries")

There are 38 unique countries


**Data Cleaning**

In [87]:
# Drop if Customer ID is null
df = df.dropna(subset='Customer ID')
df['Customer ID'] = df['Customer ID'].astype(str)

df = df[df['Quantity'] > 0]
df['TotalSpend'] = df['Quantity'] * df['Price']
df = df.drop_duplicates()
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
print(f'Cleaned Data: {df.shape[0]} rows.')

Cleaned Data: 392733 rows.


In [72]:
df.describe()

,Quantity,Price,TotalSpend
count,392733.000000,392733.000000,392733.000000
mean,13.153687,3.125633,22.629183
std,181.588189,22.240710,311.083069
min,1.000000,0.000000,0.000000
25%,2.000000,1.250000,4.950000
50%,6.000000,1.950000,12.390000
75%,12.000000,3.750000,19.800000
max,80995.000000,8142.750000,168469.600000


In [73]:
df.duplicated().sum()

np.int64(0)

In [94]:
# Set a reference date (the day after the last data point in the dataset)
# In a real live system, this would be 'today'.
snapshot_date = df['InvoiceDate'].max() + pd.Timedelta(days=1)

# Aggregate data by Customer ID
customers = df.groupby(['Customer ID']).agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days, #Recency
    'Invoice': 'count',                                      #Frequency
    'TotalSpend': 'sum'                                      #Monetary
})

#Rename columns to be clear
customers.rename(columns={
    'InvoiceDate': 'Recency',
    'Invoice': 'Frequency',
    'TotalSpend': 'TotalSpend'
}, inplace=True)

print(f'We have profiles for {customers.shape[0]} unique customers')
customers.head()

We have profiles for 4339 unique customers


,Recency,Frequency,TotalSpend
Customer ID,,,
12346,326,1,77183.60
12347,2,182,4310.00
12348,75,31,1797.24
12349,19,73,1757.55
12350,310,17,334.40


Timestamp('2011-12-10 12:50:00')